In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-e_l4q2w2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-e_l4q2w2
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=bdef18b087392175aa8f77e983e0d8590680b8c3350536d170cc1da49e9262c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8jueu1kq/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import whisper
import json
import os

# 👉 CHANGE YOUR AUDIO FOLDER PATH HERE (FULL PATH!)
AUDIO_DIR = "/content/drive/MyDrive/RAG_project/audios"   # Example
JSON_DIR = "/content/drive/MyDrive/RAG_project/json"     # Output Folder

# Create output folder if it doesn't exist
os.makedirs(JSON_DIR, exist_ok=True)

# Load Whisper model
model = whisper.load_model("large-v2")

# List all audio files in the folder
audios = [f for f in os.listdir(AUDIO_DIR) if f.endswith(".mp3")]

for audio in audios:
    audio_path = os.path.join(AUDIO_DIR, audio)

    number = audio.split("_")[0]
    title = audio.split("_")[1][:-4]

    print(f"Processing → {number} : {title}")

    # Transcribe (Hindi audio → English translation)
    result = model.transcribe(
        audio=audio_path,
        language="hi",
        task="translate",
        word_timestamps=False
    )

    # Build chunks
    chunks = []
    for segment in result["segments"]:
        chunks.append({
            "number": number,
            "title": title,
            "start": segment["start"],
            "end": segment["end"],
            "text": segment["text"]
        })

    # Final structured output
    chunks_with_metadata = {
        "chunks": chunks,
        "text": result["text"]
    }

    # Save JSON to Google Drive
    json_path = os.path.join(JSON_DIR, f"{audio}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(chunks_with_metadata, f, ensure_ascii=False, indent=2)

    print(f"✔ Saved JSON → {json_path}")
